<a href="https://colab.research.google.com/github/prp20/tensorflow_learning/blob/main/practice_notebooks/practice_flowers_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d utkarshsaxenadn/flower-classification-5-classes-roselilyetc

100% 946M/949M [00:06<00:00, 140MB/s]
100% 949M/949M [00:06<00:00, 144MB/s]


In [ ]:
! unzip flower-classification-5-classes-roselilyetc.zip -d flower_dataset

Streaming output truncated to the last 5000 lines.
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1500).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1501).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1502).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1503).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1504).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1505).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1506).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classification/Training Data/Lily/Lily (1507).jpeg  
  inflating: flower_dataset/Flower Classification/Flower Classificati

In [ ]:
TRAIN_DIR = "/content/flower_dataset/Flower_Classification_V2/V2/Training"
VAL_DIR = "/content/flower_dataset/Flower_Classification_V2/V2/Validation"
TEST_DIR = "/content/flower_dataset/Flower_Classification_V2/V2/Testing"

In [ ]:
import os
import shutil
import pandas as pd
import random
import numpy as np

import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201

import cv2

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def blur_preprocessing(img):
    return cv2.blur(img, (5, 5))

tf.random.set_seed(99)

## Initalize Image Data Generator with Augmentation
train_img_data_generator = ImageDataGenerator(rescale=1/255,
                                        rotation_range = 180,
                                        horizontal_flip = True,
                                        vertical_flip = True,
                                        preprocessing_function=blur_preprocessing
                                       )
val_image_data_generator = ImageDataGenerator(rescale = 1/255)
test_image_data_generator = ImageDataGenerator(rescale = 1/255)
## Recreate datasets from dataframe
train_data_multi = train_img_data_generator.flow_from_directory(directory=TRAIN_DIR,
                                                    target_size=(256, 256),
                                                    class_mode='categorical',
                                                    batch_size=32,
                                                    shuffle=True,
                                                    seed=42)

val_data_multi = val_image_data_generator.flow_from_directory(directory=VAL_DIR,
                                                    target_size=(256, 256),
                                                    class_mode='categorical',
                                                    batch_size=32,
                                                    shuffle=True,
                                                    seed=42)
test_data_multi = val_image_data_generator.flow_from_directory(directory=TEST_DIR,
                                                    target_size=(256, 256),
                                                    class_mode='categorical',
                                                    batch_size=32,
                                                    shuffle=True,
                                                    seed=42)

Found 15000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Found 2355 images belonging to 10 classes.


In [ ]:
# learning rate decay
learning_rate_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0003, 
                                                                         decay_steps=2, 
                                                                         decay_rate=0.97, 
                                                                         staircase=False)

In [ ]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs=None):
        self.patience = 3
        self.best = 0
        self.wait = 0
    
    def on_epoch_end(self, epoch, logs=None):
        if np.greater(logs["val_accuracy"], self.best):
            self.wait = 0
            self.best = logs["val_accuracy"]
        else:
            self.wait +=1
            if self.wait >= self.patience:
                print(f"Stopping Training. Validation accuracy hasn't improved >= {self.patience} times")
                self.model.stop_training=True

In [ ]:
def get_callbacks(model_name):
  callbacks = []
  MC = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only='True', verbose=1)
  ES = EarlyStoppingCallback()
  return [MC, ES]

In [ ]:
model_configs = dict()

def cfg_model_run(model, history, test_ds):
    return {"model": model, "history" : history, "test_ds": test_ds}

In [ ]:
def run_model(model_name, model_func, model_configs, epochs, classes):

    model = model_func(classes, trainable_weights = False)

    model_hist = model.fit(train_data_multi, validation_data=val_data_multi, epochs=epochs, steps_per_epoch = 128, callbacks= get_callbacks(model.name))
    model.save(f"saved_models/{model_name}")
    model_configs[model_name] = cfg_model_run(model, model_hist, test_data_multi)
    return test_data_multi

In [ ]:
def create_densenet_model(classes, trainable_weights=False, weights_path=None):
    tf.keras.backend.clear_session()
    dense_net = DenseNet201(input_shape=(256, 256, 3), weights="imagenet", include_top=False)
    for layer in dense_net.layers:
        layer.trainable=trainable_weights
    model = tf.keras.models.Sequential([dense_net,
                                        tf.keras.layers.GlobalAveragePooling2D(),
                                        tf.keras.layers.Dense(128, activation='relu'),
                                        tf.keras.layers.Dropout(0.3),
                                        tf.keras.layers.Dense(classes, activation='softmax')
                                ], name="densenet201")
    if weights_path:
        model.load_weights(weights_path)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_scheduler)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

def create_vgg16_model(classes, trainable_weights=False, weights_path=None):
    tf.keras.backend.clear_session()
    vgg16 = VGG16(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')
    for layer in vgg16.layers:
      layer.trainable=trainable_weights
    
    model = tf.keras.models.Sequential(name="vgg16")
    # initialize EfficientNetB3 model with input shape as (300,300,3)
    model.add(vgg16)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(256, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(classes, activation = 'softmax'))
    if weights_path:
        model.load_weights(weights_path)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_scheduler)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

def create_inception_model(classes, trainable_weights=False, weights_path=None):
    tf.keras.backend.clear_session()
    inception = InceptionV3(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')
    for layer in inception.layers:
      layer.trainable=trainable_weights
    
    model = tf.keras.models.Sequential(name="inception")
    # initialize EfficientNetB3 model with input shape as (300,300,3)
    model.add(inception)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(256, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(classes, activation = 'softmax'))
    if weights_path:
        model.load_weights(weights_path)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_scheduler)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.applications import ResNet50

def create_resnet_model(classes, trainable_weights=False, weights_path=None):
    tf.keras.backend.clear_session()
    resnet = ResNet50(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')
    for layer in resnet.layers:
      layer.trainable=trainable_weights
    
    model = tf.keras.models.Sequential(name="resnet")
    model.add(resnet)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(256, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(classes, activation = 'softmax'))
    if weights_path:
        model.load_weights(weights_path)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_scheduler)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.applications import EfficientNetB3, EfficientNetB5

def create_efficientnet_model(classes, trainable_weights=False, weights_path=None):
    tf.keras.backend.clear_session()
    efficientnet = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')
    for layer in efficientnet.layers:
      layer.trainable=trainable_weights
    
    model = tf.keras.models.Sequential(name="efficientnet")
    
    model.add(efficientnet)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(256, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(classes, activation = 'softmax'))
    if weights_path:
        model.load_weights(weights_path)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_scheduler)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model_configs=dict()
# run_model("Densenet", create_densenet_model, model_configs, epochs=25, classes = 10)
# run_model("VGG16", create_vgg16_model, model_configs, epochs=25, classes = 10)
# run_model("Inception", create_inception_model, model_configs, epochs=25, classes = 10 )
# run_model("ResNet50", create_resnet_model, model_configs, epochs=25, classes = 10)
run_model("EfficientNet", create_efficientnet_model, model_configs, epochs=25, classes = 10)

TypeError: ignored

In [ ]:
train_data_multi.class_indices

{'Aster': 0,
 'Daisy': 1,
 'Iris': 2,
 'Lavender': 3,
 'Lily': 4,
 'Marigold': 5,
 'Orchid': 6,
 'Poppy': 7,
 'Rose': 8,
 'Sunflower': 9}